# Importation

In [2]:
import numpy as np
import pandas as pd
import json

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

# Création de données

In [43]:
path = "~/hfactory_magic_folders/hi__paris_hackathon/building_energy_efficiency/datasets/"
df = pd.read_csv(path + "test/test_features_sent.csv")
df

/opt/conda/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3135: DtypeWarning: Columns (19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,level_0,additional_heat_generators,additional_water_heaters,altitude,area_code,balcony_depth,bearing_wall_material,building_category,building_class,building_height_ft,...,wall_insulation_type,water_heaters,water_heating_energy_source,water_heating_type,window_filling_type,window_frame_material,window_glazing_type,window_heat_retention_factor,window_orientation,window_thermal_conductivity
0,1371914,[],NaN,249.34384,615,NaN,BRICKS,[],[],19.685040,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,1.067688,"[north,sud]",21.349539
1,618756,[],NaN,104.98688,304,NaN,STONE,[],[],28.215224,...,non insulated,[electric storage tank],electricity,individual,argon or krypton,pvc,double glazing,1.149140,[west],13.963716
2,871032,[],NaN,72.17848,204,NaN,BRICKS,[individual house],[individual],25.918636,...,non insulated,[indeterminate gas boiler],gas,individual,dry air,pvc,double glazing,1.084204,"[north,sud]",14.649374
3,1430640,[oil stove or insert],NaN,800.52496,858,NaN,BRICKS,[individual house],[individual],15.748032,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,1.067688,"[est,west]",15.347292
4,86306,[wood stove or insert],NaN,1010.49872,671,NaN,GRITSTONE,[individual house],[individual],0.000000,...,internal,[electric storage tank],electricity,individual,dry air,pvc,double glazing,3.592767,[],17.512631
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494935,845834,[],NaN,NaN,467,NaN,OTHER,[],[],NaN,...,internal,[electric storage tank],electricity,individual,dry air,wood,double glazing,3.887723,[],17.512631
494936,279675,[],NaN,377.29660,748,NaN,CONCRETE,[],[],15.748032,...,internal,[electric storage tank],electricity,individual,argon or krypton,pvc,double glazing,1.084204,[north],13.963716
494937,811813,[],NaN,830.05252,838,NaN,CONCRETE,[condo],[2 to 11],36.089240,...,internal,[indeterminate gas boiler],gas,individual,argon or krypton,pvc,double glazing,1.034308,[est],8.947385
494938,369033,[wood stove or insert],NaN,288.71392,752,NaN,AGGLOMERATE,[individual house],[individual],14.763780,...,internal,[electric storage tank],electricity,individual,dry air,pvc,double glazing,1.084204,"[est,west]",13.963716


In [45]:
df_level_0 = df[['level_0']]
df_level_0

,level_0
0,1371914
1,618756
2,871032
3,1430640
4,86306
...,...
494935,845834
494936,279675
494937,811813
494938,369033


# Data creation function

In [7]:
df.consumption_measurement_date = df.consumption_measurement_date.apply(lambda x: str(int(x[5:7])//3))
df.outer_wall_thickness = df.outer_wall_thickness.apply(lambda x: float(x) if isinstance(x, float) or isinstance(x, int) else float(x.split()[0]))
df["building_period"] = df["building_period"].apply(lambda x: ">1989" if x in ["1989-1999", "2000-2005", '2006-2012', ">2012"] else x)


In [8]:
liste_str = ['balcony_depth', 'building_period', 'building_type', 'building_use_type_description',
 'clay_risk_level', 'has_air_conditioning', 'has_balcony', 'heating_type', 'is_crossing_building',
 'lower_floor_adjacency_type', 'lower_floor_material', 'radon_risk_level', 'solar_heating',
 'solar_water_heating', 'thermal_inertia', 'upper_floor_adjacency_type', 'water_heating_type',
 'window_filling_type', 'window_glazing_type', 'consumption_measurement_date']

In [9]:
liste_int = ['altitude', 'area_code', 'building_height_ft',
       'building_total_area_sqft', 'building_use_type_code', 'building_year', 'living_area_sqft',
       'lowe_floor_thermal_conductivity', 'nb_dwellings',
       'outer_wall_thermal_conductivity', 'post_code', 'upper_floor_thermal_conductivity',
       'window_heat_retention_factor', 'window_thermal_conductivity', 'outer_wall_thickness', 
         'percentage_glazed_surfaced']

In [10]:
mediane = pd.read_csv("median.csv", header=None, index_col=0, squeeze=True)
df[liste_int] = df[liste_int].fillna(mediane)

In [11]:
liste = liste_str + liste_int
df = df[liste]

In [12]:
def transform(df, liste):
    for x in liste:
        list_value = [y for y in df[x].unique() if y is not np.nan]
        dict_value = {y:y for y in list_value}
        df[x] = df[x].map(dict_value)
        df = pd.get_dummies(df, columns=[x], prefix=f'{x}-', prefix_sep='')
    return df

df = transform(df, liste_str)

/tmp/ipykernel_192582/1076129779.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[x] = df[x].map(dict_value)


In [30]:
liste_col = ['altitude', 'area_code', 'building_height_ft',
       'building_total_area_sqft', 'building_use_type_code', 'building_year',
       'living_area_sqft', 'lowe_floor_thermal_conductivity', 'nb_dwellings',
       'outer_wall_thermal_conductivity', 'post_code',
       'upper_floor_thermal_conductivity', 'window_heat_retention_factor',
       'window_thermal_conductivity', 'outer_wall_thickness',
       'percentage_glazed_surfaced', 'balcony_depth-1 <= … < 2',
       'balcony_depth-2 <= … < 3', 'balcony_depth-3 <=', 'balcony_depth-< 1 m',
       'building_period-1949-1970', 'building_period-1970-1988',
       'building_period-<1948', 'building_period->1989',
       'building_period-bad sup', 'building_type-Flat', 'building_type-House',
       'building_type-Multiple dwellings', 'building_type-Non-residential',
       'building_type-Shopping centres',
       'building_use_type_description-Commercial & Other',
       'building_use_type_description-Residential multi-family',
       'building_use_type_description-Residential single-family',
       'clay_risk_level-high', 'clay_risk_level-low', 'clay_risk_level-medium',
       'has_air_conditioning-False', 'has_air_conditioning-True',
       'has_balcony-False', 'has_balcony-True', 'heating_type-collective',
       'heating_type-individual', 'is_crossing_building-90° crossing (weak)',
       'is_crossing_building-all through crossing (weak)',
       'is_crossing_building-crossing east west',
       'is_crossing_building-crossing north south',
       'is_crossing_building-east-west crossing (weak)',
       'is_crossing_building-north-south crossing (weak)',
       'is_crossing_building-not through', 'is_crossing_building-through 90°',
       'is_crossing_building-through all way',
       'lower_floor_adjacency_type-ADJACENT BUILDING',
       'lower_floor_adjacency_type-BURIED_WALL',
       'lower_floor_adjacency_type-FULL_EARTH',
       'lower_floor_adjacency_type-LNC', 'lower_floor_adjacency_type-OUTSIDE',
       'lower_floor_adjacency_type-SANITARY_VACTILE',
       'lower_floor_material-Insulated joist floor',
       'lower_floor_material-Wooden floor on metal joists',
       'lower_floor_material-brick or rubble wall joists',
       'lower_floor_material-concrete slab',
       'lower_floor_material-floor between metal joists with or without infill',
       'lower_floor_material-floor between wooden joists with or without infill',
       'lower_floor_material-floor with or without infill',
       'lower_floor_material-heavy floor, such as clay floor joists, concrete beams',
       'lower_floor_material-joist on metal joists',
       'lower_floor_material-non-differing party floor',
       'lower_floor_material-shingles and infill',
       'lower_floor_material-wooden floor on wooden joists',
       'radon_risk_level-high', 'radon_risk_level-low',
       'radon_risk_level-medium', 'solar_heating-False', 'solar_heating-True',
       'solar_water_heating-False', 'solar_water_heating-True',
       'thermal_inertia-high', 'thermal_inertia-low', 'thermal_inertia-medium',
       'thermal_inertia-very high',
       'upper_floor_adjacency_type-ADJACENT_BUILDING',
       'upper_floor_adjacency_type-EXTERNAL',
       'upper_floor_adjacency_type-INNER_WALL',
       'upper_floor_adjacency_type-LNC', 'water_heating_type-collective',
       'water_heating_type-individual', 'window_filling_type-argon or krypton',
       'window_filling_type-dry air', 'window_glazing_type-double glazing',
       'window_glazing_type-glass block or polycarbonate',
       'window_glazing_type-overglazing', 'window_glazing_type-single glazing',
       'window_glazing_type-triple glazing', 'consumption_measurement_date-0',
       'consumption_measurement_date-1', 'consumption_measurement_date-2',
       'consumption_measurement_date-3', 'consumption_measurement_date-4']

In [31]:
inputs_with_num = df[liste_col].to_numpy()
inputs_with_num.shape

(494940, 98)

## features

In [32]:
test_features = inputs_with_num

# Reload Model

In [33]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [34]:
MODEL_PATH = "./tf_model7"
model = tf.keras.models.load_model(MODEL_PATH)

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizatio  (None, 98)               197       
 n)                                                              
                                                                 
 dropout (Dropout)           (None, 98)                0         
                                                                 
 dense (Dense)               (None, 64)                6336      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 16)                528       
                                                        

# Predict

In [36]:
predictions = model.predict(test_features).T[0]
predictions

array([226.33867, 238.82388, 253.70264, ..., 243.26698, 196.0737 ,
       175.68503], dtype=float32)

# Création submit

In [46]:
df_level_0["energy_consumption_per_annum"] = predictions

/tmp/ipykernel_192582/3492476888.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_level_0["energy_consumption_per_annum"] = predictions


In [47]:
df_level_0.to_csv("Submission4.csv", index=False)

# Analyse

In [48]:
import sklearn.metrics

In [49]:
df2 = pd.read_csv("Submission2.csv")
df3 = pd.read_csv("Submission3.csv")

In [50]:
df_level_0

,level_0,energy_consumption_per_annum
0,1371914,226.338669
1,618756,238.823883
2,871032,253.702637
3,1430640,199.150726
4,86306,216.095413
...,...,...
494935,845834,300.238831
494936,279675,135.256363
494937,811813,243.266983
494938,369033,196.073700


In [51]:
sklearn.metrics.explained_variance_score(
    df2.energy_consumption_per_annum.to_list(), 
    df_level_0.energy_consumption_per_annum.to_list()
)

0.16242206846283136

In [52]:
sklearn.metrics.explained_variance_score(
    df3.energy_consumption_per_annum.to_list(), 
    df_level_0.energy_consumption_per_annum.to_list()
)

0.8031434591333316